# Estimation de la consommation par habitant

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import math
import random
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.cluster import KMeans 

In [2]:
BASE_DIR ='C:/Users/d/Desktop/Programme du projet sur CNN pauvreté/'
RANDOM_SEED = 7 # for reproducibility
COUNTRIES_DIR = os.path.join(BASE_DIR, 'Data', 'countries')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')
FIGURES_DIR = os.path.join(BASE_DIR, 'figures')

#### Fonction pour fusionner deux DataFrames sur la latitude et la longitude

def merge_on_lat_lon(df1, df2, keys=['cluster_lat', 'cluster_lon'], how='inner'): a pour but de Fusionner deux DataFrames df1 et df2 sur des colonnes spécifiques qui représentent des coordonnées géographiques (latitude et longitude).
La fusion basée sur des valeurs flottantes (comme les coordonnées géographiques) peut conduire à des imprécisions dues à la représentation en virgule flottante. Ce script contourne le problème en convertissant ces valeurs flottantes en entiers avant la fusion.
Les colonnes de latitude et longitude (keys) sont multipliées par 10000 et converties en entiers. Cette opération vise à préserver la précision tout en évitant les erreurs liées aux opérations sur les flottants.
Les colonnes originales de latitude et longitude sont supprimées dans df2 pour éviter les doublons après la fusion.
Une fusion (pd.merge) est effectuée sur les nouvelles colonnes entières avec l'option how permettant de spécifier le type de jointure (par défaut inner).
Les colonnes temporaires utilisées pour la fusion sont supprimées avant de retourner le DataFrame fusionné.

In [3]:
def merge_on_lat_lon(df1, df2, keys=['cluster_lat', 'cluster_lon'], how='inner'):
    """
    Fusionne deux DataFrames sur les colonnes de latitude et longitude.
    Nécessaire car la fusion directe sur des nombres flottants peut introduire des erreurs dues à la précision.
    """
    # Crée des copies des DataFrames pour éviter de modifier les originaux
    df1 = df1.copy()
    df2 = df2.copy()
    
    # Convertit les valeurs de latitude et longitude en entiers en multipliant par 10000
    # pour éviter les problèmes de précision avec les nombres flottants
    df1['merge_lat'] = (10000 * df1[keys[0]]).astype(int)
    df1['merge_lon'] = (10000 * df1[keys[1]]).astype(int)
    
    df2['merge_lat'] = (10000 * df2[keys[0]]).astype(int)
    df2['merge_lon'] = (10000 * df2[keys[1]]).astype(int)
    
    # Supprime les colonnes originales de latitude et longitude de df2 pour éviter les doublons
    df2.drop(keys, axis=1, inplace=True)
    
    # Fusionne les DataFrames sur les colonnes d'entiers de latitude et longitude
    merged = pd.merge(df1, df2, on=['merge_lat', 'merge_lon'], how=how)
    
    # Supprime les colonnes de fusion temporaires
    merged.drop(['merge_lat', 'merge_lon'], axis=1, inplace=True)
    
    return merged

#### Fonction pour assigner des groupes (clusters) à partir des coordonnées

def assign_groups(df, k, random_seed=7): a pour but d'assigner les points dans df (représentés par les colonnes cluster_lat, cluster_lon) à k groupes (ou clusters) en utilisant l'algorithme K-Means.
Un seed aléatoire est fixé pour garantir la reproductibilité des résultats.
L'algorithme K-Means est initialisé avec k centres et appliqué aux coordonnées de cluster.
La fonction retourne les étiquettes de cluster pour chaque point et les centres des clusters identifiés.

In [4]:
def assign_groups(df, k, random_seed=7):
    """
    Attribue les points dans df (colonnes cluster_lat, cluster_lon) à k groupes (clusters)
    en utilisant l'algorithme K-Means, et retourne également les centres des clusters.
    """
    # Fixe la graine aléatoire pour la reproductibilité
    np.random.seed(random_seed)
    
    # Initialise KMeans avec k clusters et applique l'algorithme sur les coordonnées
    km = KMeans(k)
    cluster_labels, cluster_centers = km.fit_predict(df[['cluster_lat', 'cluster_lon']]), km.cluster_centers_
    
    return cluster_labels, cluster_centers


#### Fonction pour exécuter une validation croisée randomisée

def run_randomized_cv(X, y, k=5, k_inner=5, random_seed=7, points=10, alpha_low=1, alpha_high=5, to_print=False):
a pour but d'exécuter une validation croisée randomisée sur les ensembles de données X et y, typiquement pour évaluer la performance d'un modèle de régression.
-Initialise un seed aléatoire pour la reproductibilité.
-Génère une grille d'hyperparamètres alphas logarithmiquement espacés entre alpha_low et alpha_high.
-Divise les données en k folds pour la validation croisée externe, et potentiellement en k_inner folds pour une validation croisée interne (non montrée dans le snippet).
-Pour chaque fold externe, sélectionne le meilleur hyperparamètre basé sur la performance dans la validation croisée interne, puis évalue ce modèle sur le fold de test externe.
-Agrège et retourne la performance moyenne (R²) et les prédictions (y_hat).

In [5]:
def run_randomized_cv(X, y, k=5, k_inner=5, random_seed=7, points=10, alpha_low=1, alpha_high=5, to_print=False):
    """
    Exécute une validation croisée randomisée sur X et y.
    Retourne le coefficient de détermination R² moyen et les prédictions y_hat.
    """
    # Fixe la graine aléatoire pour la reproductibilité
    np.random.seed(random_seed)
    
    # Génère une séquence logarithmique d'alphas entre alpha_low et alpha_high
    alphas = np.logspace(alpha_low, alpha_high, points)
    
    r2s = []  # Liste pour stocker les scores R² de chaque fold
    y_hat = np.zeros_like(y)  # Initialise un tableau pour les prédictions
    
    # Initialise KFold pour la division des données
    kf = KFold(n_splits=k, shuffle=True)
    
    fold = 0  # Compteur de fold
    for train_idx, test_idx in kf.split(X):
        # Évalue chaque fold et récupère le score R² et les prédictions
        r2, y_p = evaluate_fold(X, y, train_idx, test_idx, k_inner, alphas, to_print)
        r2s.append(r2)
        y_hat[test_idx] = y_p
        fold += 1
    
    # Calcule et retourne le R² moyen et les prédictions complètes
    return np.mean(r2s), y_hat


In [6]:
pd.read_csv(os.path.join(BASE_DIR,'Data','Processed',"image_download_actual.csv"))
#pd.read_csv(os.path.join(BASE_DIR,'Data','Processed',"image_download_locs.csv"))

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,pcexp,nightlights,nightlights_bin,is_train
0,5.4638471458443405_-4.05723036205941_5.4641465...,5.463847,-4.057230,5.464147,-4.056781,518465.0,181.0,1,True
1,5.46399686505836_-4.05723036205941_5.464146584...,5.463997,-4.057230,5.464147,-4.056781,518465.0,181.0,1,False
2,5.46414658427238_-4.05723036205941_5.464146584...,5.464147,-4.057230,5.464147,-4.056781,518465.0,181.0,1,True
3,5.4642963034864005_-4.05723036205941_5.4641465...,5.464296,-4.057230,5.464147,-4.056781,518465.0,181.0,1,True
4,5.46444602270042_-4.05723036205941_5.464146584...,5.464446,-4.057230,5.464147,-4.056781,518465.0,181.0,1,True
...,...,...,...,...,...,...,...,...,...
2929,6.859985734133001_-6.430992092734161_6.8598360...,6.859986,-6.430992,6.859836,-6.430842,611718.1,186.0,2,False
2930,6.86013545334702_-6.430992092734161_6.85983601...,6.860135,-6.430992,6.859836,-6.430842,611718.1,186.0,2,True
2931,6.86028517256104_-6.430992092734161_6.85983601...,6.860285,-6.430992,6.859836,-6.430842,611718.1,186.0,2,True
2932,6.85938685727692_-6.43084237352014_6.859836014...,6.859387,-6.430842,6.859836,-6.430842,611718.1,186.0,2,True


In [7]:
COUNTRIES_DIR = os.path.join(BASE_DIR, 'Data', 'countries')
PROCESSED_DIR = os.path.join(BASE_DIR, 'Data', 'processed')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')
CNN_TRAIN_IMAGE_DIR = os.path.join(BASE_DIR, 'Data', 'cnn_images')
CNN_SAVE_DIR = os.path.join(BASE_DIR, 'models')
CNN_DIR = os.path.join(BASE_DIR, 'models', 'trained_model.pt')

def load_country(country='CIV_2018'):
    '''
    Organizes the country's dataframe so that each index corresponds to the index in the cluster features
    Returns the cluster features and the organized dataframe
    '''
    # Utilisation de la variable `country` pour construire le chemin
    country_processed_dir = os.path.join(PROCESSED_DIR)  # Pas de changement nécessaire si tous les pays utilisent le même dossier processed
    country_results_dir = os.path.join(RESULTS_DIR, country, 'cnn')
    x = np.load(os.path.join(country_results_dir, 'cluster_feats.npy'))
    cluster_list = pickle.load(open(os.path.join(country_results_dir, 'cluster_order.pkl'), 'rb'))
    cluster_list = pd.DataFrame.from_records(cluster_list, columns=['cluster_lat', 'cluster_lon'])
    cluster_list['feat_index'] = np.arange(len(cluster_list))
    
    # Vérifier'image_download_actual.csv' contient les informations nécessaires
    df_clusters = pd.read_csv(os.path.join(country_processed_dir, 'image_download_actual.csv'))  
    assert len(df_clusters) == len(cluster_list)

    df = merge_on_lat_lon(df_clusters, cluster_list, keys=['cluster_lat', 'cluster_lon'])
    assert len(df) == len(df_clusters) == len(cluster_list)
    df.sort_values('feat_index', ascending=True, inplace=True)
    return x, df


In [29]:
len(pd.DataFrame.from_records(pickle.load(open(os.path.join(os.path.join(RESULTS_DIR, 'CIV_2018', 'cnn'), 'cluster_order.pkl'), 'rb')), columns=['cluster_lat', 'cluster_lon']))

565

In [30]:
pd.read_csv(os.path.join(COUNTRIES_DIR, 'civ_2018', 'DataCIV.csv'))

,country,year,hhid,grappe,menage,vague,zae,region,milieu,milieu2,...,interview__key,interview__id,GPS__Latitude,GPS__Longitude,GPS__Accuracy,GPS__Altitude,GPS__Timestamp,nom_prenom_cm,localisation_menage,nightlights
0,CIV,2018,101.0,1,1,1,ABIDJAN,AUTONOME D'ABIDJAN,Urbain,Abidjan urbain,...,37-12-91-96,2eb3d29ba9814aa9bf99d6908559e7d2,5.340541,-4.026030,46.0,39.0,2018-10-29T17:56:23,NaN,NaN,175.0
1,CIV,2018,102.0,1,2,1,ABIDJAN,AUTONOME D'ABIDJAN,Urbain,Abidjan urbain,...,66-72-76-42,99990d719d6243a38fad96a4d7477742,5.340402,-4.025834,47.0,28.0,2018-10-29T17:54:35,NaN,NaN,0.0
2,CIV,2018,103.0,1,3,1,ABIDJAN,AUTONOME D'ABIDJAN,Urbain,Abidjan urbain,...,00-21-02-40,6cf633ef989c4a6a93ec483de1fd8193,5.340151,-4.025953,35.0,68.0,2018-10-29T17:51:50,NaN,NaN,0.0
3,CIV,2018,104.0,1,4,1,ABIDJAN,AUTONOME D'ABIDJAN,Urbain,Abidjan urbain,...,69-94-30-00,67feab4e35d141679ec4b867a59cba19,5.340500,-4.026202,32.0,59.0,2018-10-29T19:48:39,NaN,NaN,175.0
4,CIV,2018,105.0,1,5,1,ABIDJAN,AUTONOME D'ABIDJAN,Urbain,Abidjan urbain,...,92-17-10-43,dc1f1fa87d8e433e8891a4e55a8baca0,5.339820,-4.025066,49.0,72.0,2018-10-29T18:45:09,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12975,CIV,2019,108408.0,1084,8,2,CENTRE,MORONOU,Rural,Rural,...,50-03-51-41,103cff29baaf4405863e1043897aa6dc,6.546733,-4.508211,7.0,134.0,2019-06-16T19:42:05,NaN,NaN,0.0
12976,CIV,2019,108409.0,1084,9,2,CENTRE,MORONOU,Rural,Rural,...,82-42-90-66,e3a3deb8965e45c18018d1e5d103d914,6.549112,-4.505840,36.0,131.0,2019-06-17T08:10:57,NaN,NaN,0.0
12977,CIV,2019,108410.0,1084,10,2,CENTRE,MORONOU,Rural,Rural,...,21-28-94-39,45397589c6d44a478051e4089a884c97,6.551556,-4.507530,29.0,118.0,2019-06-16T09:30:14,NaN,NaN,0.0
12978,CIV,2019,108411.0,1084,11,2,CENTRE,MORONOU,Rural,Rural,...,24-86-66-78,505343592b0f4b8b8ede9980f714b8ab,6.552345,-4.507551,19.0,157.0,2019-06-16T10:18:13,NaN,NaN,0.0


In [9]:
'''
COUNTRIES_DIR = os.path.join(BASE_DIR, 'Data', 'countries')
PROCESSED_DIR = os.path.join(BASE_DIR, 'Data', 'processed')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')
CNN_TRAIN_IMAGE_DIR = os.path.join(BASE_DIR, 'Data', 'cnn_images')
CNN_SAVE_DIR = os.path.join(BASE_DIR, 'models')
CNN_DIR = os.path.join(BASE_DIR, 'models', 'trained_model.pt')

def load_country(country='CIV_2018'):
    
    Organizes the country's dataframe so that each index corresponds to the index in the cluster features
    Returns the cluster features and the organized dataframe
    
    # Utilisation de la variable `country` pour construire le chemin
    country_processed_dir = os.path.join(PROCESSED_DIR)  # Pas de changement nécessaire si tous les pays utilisent le même dossier processed
    country_results_dir = os.path.join(RESULTS_DIR, country, 'cnn')  # Correction pour utiliser la variable `country`
    x = np.load(os.path.join(country_results_dir, 'cluster_feats.npy'))
    cluster_list = pickle.load(open(os.path.join(country_results_dir, 'cluster_order.pkl'), 'rb'))
    cluster_list = pd.DataFrame.from_records(cluster_list, columns=['cluster_lat', 'cluster_lon'])
    cluster_list['feat_index'] = np.arange(len(cluster_list))
    
    # Assurez-vous que 'image_download_actual.csv' contient les informations nécessaires
    df_clusters =pd.read_csv(os.path.join(COUNTRIES_DIR, 'civ_2018', 'DataCIV.csv'))  # Assurez-vous que ce fichier existe et est correct
    assert len(df_clusters) == len(cluster_list)

    df = merge_on_lat_lon(df_clusters, cluster_list, keys=['cluster_lat', 'cluster_lon'])
    assert len(df) == len(df_clusters) == len(cluster_list)
    df.sort_values('feat_index', ascending=True, inplace=True)
    return x, df
'''

"\nCOUNTRIES_DIR = os.path.join(BASE_DIR, 'Data', 'countries')\nPROCESSED_DIR = os.path.join(BASE_DIR, 'Data', 'processed')\nRESULTS_DIR = os.path.join(BASE_DIR, 'results')\nCNN_TRAIN_IMAGE_DIR = os.path.join(BASE_DIR, 'Data', 'cnn_images')\nCNN_SAVE_DIR = os.path.join(BASE_DIR, 'models')\nCNN_DIR = os.path.join(BASE_DIR, 'models', 'trained_model.pt')\n\ndef load_country(country='CIV_2018'):\n    \n    Organizes the country's dataframe so that each index corresponds to the index in the cluster features\n    Returns the cluster features and the organized dataframe\n    \n    # Utilisation de la variable `country` pour construire le chemin\n    country_processed_dir = os.path.join(PROCESSED_DIR)  # Pas de changement nécessaire si tous les pays utilisent le même dossier processed\n    country_results_dir = os.path.join(RESULTS_DIR, country, 'cnn')  # Correction pour utiliser la variable `country`\n    x = np.load(os.path.join(country_results_dir, 'cluster_feats.npy'))\n    cluster_list = 

In [10]:
load_country(country='CIV_2018')

AssertionError: 